На основании базы знаний с ПРАВИЛАМИ СТРАХОВАНИЯ ОТВЕТСТВЕННОСТИ АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ создайте нейро-консультанта, который бы отвечал на вопросы клиентов по информации, содержащейся в представленном документе. Напишите промпт в system и user самостоятельно. Добейтесь того, чтобы нейро-консультант в своих ответах не ссылался на документы, переданные ему для ответа.

Документ с базой знаний можно найти тут: https://docs.google.com/document/d/11MU3SnVbwL_rM-5fIC14Lc3XnbAV4rY1Zd_kpcMuH4Y

In [ ]:
!pip install faiss-cpu langchain==0.0.271 openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
import os
import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
# функция для загрузки документа по ссылке из Google Drive
def load_document_text(url: str) -> str:
    # Извлечение идентификатора документа из URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Загрузить документ в виде обычного текста
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# База знаний
data_from_url= load_document_text('https://docs.google.com/document/d/11MU3SnVbwL_rM-5fIC14Lc3XnbAV4rY1Zd_kpcMuH4Y/edit') # заполните параметр функции

In [ ]:
# Инструкция в system
system = load_document_text('https://docs.google.com/document/d/1CGJFEwHqgMqDHgy4KJocgzMZEXCbgIuMFinoPiRIFSo/edit?usp=sharing')

In [ ]:
source_chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={}))
print("Общее количество чанков: ", len(source_chunks))

Общее количество чанков:  117


In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

In [ ]:
# Функция выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return " ".join(lines)

def answer_index(system, topic, search_index, temp=1, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=5)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос клиента: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer

Проверьте работу нейро-консультанта на сформулированных вами вопросах (достаточно 1-2)

In [ ]:
topic= "Из какого документа твои знания?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================выше уведомления подаются в письменном виде. AVN.52G 17.10.01 (применяется для страхования компаний, занимающихся обслуживанием)  
Отрывок документа №2
=====================официальном бланке с указанием занимаемой должности, периода работы, а также ежемесячного размера дохода, заверенная печатью организации и подписью уполномоченного лица) - документы, подтверждающие дополнительные расходы, необходимые для восстановления здоровья потерпевшего лица (на усиленное питание, санаторнокурортное лечение, посторонний уход, протезирование, транспортные расходы и т.п.), выполненные в соответствии с медицинскими диагнозами, и рекомендациями по восстановлению здоровья лечащего врача, подтвержденными документально. - документы, подтверждающие расходы на погребение (при причинении вреда жизни);  
Отрывок документа №3
=====================12.2.1. В случае причинения вреда жизни и здоровью фи

' Из баз интернета!'

In [ ]:
topic= "Что является объект страхования?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================2.3. Договор страхования считается заключенным в пользу потерпевших лиц (Выгодоприобретателей), которым может быть причинен вред при осуществлении Страхователем/Застрахованными лицами производственной деятельности. 
3. ОБЪЕКТ СТРАХОВАНИЯ 
3.1. Объектом страхования являются имущественные интересы Страхователя, связанные с рисками, указанными в разделе 4 настоящих Правил в отношении застрахованной по договору страхования производственной деятельности. 
4. СТРАХОВЫЕ РИСКИ И СТРАХОВЫЕ СЛУЧАИ 
4.1. Страховым риском является предполагаемое событие, на случай наступления которого проводится страхование. Страховщик принимает на страхование риски наступления гражданской ответственности Страхователя за вред жизни, здоровью и/или имуществу юридических или физических лиц, нанесенный в процессе производственной деятельности.  
Отрывок документа №2
=====================10.3. Договор страхова

' Объектом страхования являются имущественные интересы Страхователя, связанные с рисками, указанными в разделе 4 настоящих Правил в отношении застрахованной по договору  страхования производственной деятельности.'

In [ ]:
topic= "Что это значит 2488AGM00003?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================выше уведомления подаются в письменном виде. AVN.52G 17.10.01 (применяется для страхования компаний, занимающихся обслуживанием)  
Отрывок документа №2
=====================ОГОВОРКА ОБ ИСКЛЮЧЕНИИ ПРЕТЕНЗИЙ, СВЯЗАННЫХ С ВОЗДЕЙСТВИЕМ АСБЕСТА AGM248800003 «Настоящим полисом не покрываются любые претензии, прямо или косвенно связанные с указанными ниже причинами, вытекающие из них или являющиеся их следствием: Фактическое, утверждаемое или вероятное наличие асбеста в любой форме или любого материала или продукта, фактически или вероятно содержащего асбест; или 11 Любое обязательство, запрос, требование, указание или законодательное или правовое постановление в отношении страхователя или иных лиц относительно проверки на наличие, исследования, расчистки, уборки, содержания, обработки, нейтрализации, защиты от асбеста или иных действий в связи с фактическим, утверждаемым или возможны

' 2488AGM00003 - это код, который относится к оговорке об исключении асбеста в страховом полисе. Эта оговорка указывает, что любые претензии, связанные с наличием асбеста  или материалов, содержащих асбест, не покрываются данным полисом. Однако, исключение не применяется в случае претензий, вытекающих из пожара, взрыва при аварии или  других зарегистрированных происшествий с воздушным судном в полете, которые привели к эксплуатации воздушного судна в нештатном режиме.'

In [ ]:
topic= "Кто твои родители?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================использования на воздушных судах.  
Отрывок документа №2
=====================1.2.16. Третьи лица - физические или юридические лица, муниципальные образования, субъекты Российской Федерации, Российская Федерация, не имеющие договорных отношений со Страхователем, имеющие право на предъявление претензий к Страхователю о возмещении причиненного им вреда. К третьим лицам не относится сам Страхователь/ Застрахованные лица, их работники, во время выполнения ими своих служебных обязанностей, а также физические или юридические лица, выступающие от имени Страхователя. 
2. СУБЪЕКТЫ СТРАХОВАНИЯ 
2.1. Страхователями признаются юридические или физические лица, являющиеся владельцами или эксплуатантами аэропортов, поставщиками авиационных товаров и услуг, а также органы управления воздушным движением. 
2.2. Страховщик – страховая организация АО «АльфаСтрахование», действующая в соответствии 

' Человечество!'

In [ ]:
topic= "Как слетать в космос?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================использования на воздушных судах.  
Отрывок документа №2
=====================когда происходит перечисленное в пунктах (i), (ii) или (iii), то покрытие, предоставляемое настоящим Индоссаментом (если только оно не аннулировано, прекратило действие или приостановлено иным образом), продолжает действовать в отношении такого Воздушного судна до завершения его первой посадки после этого события и когда все пассажиры произвели высадку.  
Отрывок документа №3
=====================15 ОГОВОРКА ОБ ОТВЕТСТВЕННОСТИ ЗА ПРЕКРАЩЕНИЕ ЭКСПЛУАТАЦИИ ОБОРУДОВАНИЯ ПО РАСПОРЯЖЕНИЮ АВИАЦИОННЫХ ВЛАСТЕЙ (ГРАУНДИНГ) 1. От имени Страхователя оплачиваются все суммы, которые Страхователь должен уплатить в силу закона в качестве убытков от потери использования укомплектованного воздушного судна, произошедшей после доставки и приемки покупателем или покупателями или эксплуатантом или эксплуатантами такого во

' Я очень сожалею, но мне не разрешено поддержать диалог, не по делу!'

In [ ]:
topic= "Как определить убыток при приченении вреда владельцу воздушного судна?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================12.2.3. При причинении вреда владельцам/эксплуатантам ВС: - документы о регистрации происшествия с воздушным судном; Стр. - 17 - отчет комиссии по результатам расследования происшествия, включая, если это технически возможно, расшифровку полетной информации (в том числе переговоры экипажа) и полетный лист (задание на полет); - технический акт осмотра воздушного судна после его повреждения, а в случае гибели - акт о списании воздушного судна; - документы, удостоверяющие право собственности (владения, пользования) на воздушное судно или устанавливающие размер ответственности перед собственником в случае гибели или повреждения воздушного судна; - свидетельство о государственной регистрации прав на воздушное судно или его аналог предусмотренный нормативными актами; - сертификат эксплуатанта воздушного судна или его аналог предусмотренный нормативными актами; - сертификат летной год

' Для определения убытка при причинении вреда владельцу воздушного судна необходимо предоставить следующие документы: - документы о регистрации происшествия с воздушным  судном; - отчет комиссии по результатам расследования происшествия, включая расшифровку полетной информации и полетный лист; - технический акт осмотра воздушного судна  после его повреждения или акт о списании в случае гибели; - документы, удостоверяющие право собственности на воздушное судно или устанавливающие размер ответственности  перед собственником; - свидетельство о государственной регистрации прав на воздушное судно; - сертификат эксплуатанта воздушного судна; - сертификат летной годности  воздушного судна; - свидетельство о регистрации воздушного судна или его аналог.'